# Singulärwertzerlegung (Singular Value Decomposition, SVD)

Gegben: Matrix $A \in M(m \times n)$

Gesucht: Zerlegung $$A = U \Sigma V^T$$ wobei $U \in M(m \times m)$, $\Sigma \in M(m \times n)$ und $V \in M(n \times n)$

In [1]:
# input matrix
A = matrix([
[3,-2],
[-2,3]
])

In [2]:
m, n = A.dimensions()
print A, " {} x {}".format(m, n)

[ 3 -2]
[-2  3]  2 x 2


1. Eigenwerte von $A^T A$

In [3]:
ata = A.T*A; ata

[ 13 -12]
[-12  13]

In [4]:
ata.eigenvalues()

[25, 1]

2. Singulärwerte $\sigma_i = \sqrt{e_i}$ für alle $e_i \neq 0$

In [5]:
SV = [sqrt(abs(e)) for e in ata.eigenvalues() if e != 0]
sorted(SV)
SV

[5, 1]

3. Orthonormieren der Eigenvektoren von $A^T A$ um die Matrix $V$ zu erhalten.

In [6]:
ata.eigenvectors_left()
V = []
EW = []
for x in ata.eigenvectors_left():
    for y in x[1]:
        EW.append(x[0])
        V.append(vector([val for val in y]))

In [7]:
# input your own EW/EVs if they differ in position
#EW = []
#V = []

In [8]:
EW, V

([25, 1], [(1, -1), (1, 1)])

Gram-Schmidt Verfahren zur Orthonormalisierung

In [9]:
W = []
U = []
W.append((1/(V[0].norm())) * V[0])
for v in V[1:]:
    s = sum([(v.dot_product(w) * w) for w in W])
    u = v - s
    U.append(u)
    w = (1/u.norm()) * u
    W.append(w)

In [10]:
V

[(1, -1), (1, 1)]

In [11]:
W

[(1/2*sqrt(2), -1/2*sqrt(2)), (1/2*sqrt(2), 1/2*sqrt(2))]

In [12]:
U

[(1, 1)]

4. Berechnung der Spaltenvektoren $u_i$ der Matrix $U$

$$ u_i = \frac{1}{\sigma_i} A v_i $$

In [13]:
U = []
for e,v in zip(EW, W):
    if e != 0:
        u = (1/sqrt(e)) * A * v
        U.append(u)

# add an orthogonal vector if we miss some values
if len(U) != m:
    if len(U[0]) == 1:
        U.append(-1 * U[0])
    elif len(U[0]) == 2:
        U.append(vector([-1 * U[0][1], U[0][0]]))
    elif len(U[0]) == 3:
        assert len(U) == 2
        U.append(U[0].cross_product(U[1]))
    else:
        raise Exception("Can't handle dimensions > 3 in this case :'( ")

In [14]:
U

[(1/2*sqrt(2), -1/2*sqrt(2)), (1/2*sqrt(2), 1/2*sqrt(2))]

In [15]:
U = matrix(U).T
V = matrix(W).T
Vt = V.T
D = matrix([ [0] * i + [sv] + [0] * (len(SV) - i - 1) for i, sv in enumerate(SV)])
S = matrix([[0] * i + [SV[i]] + [0] * (n - i - 1) 
            if i < len(SV) 
            else [0] * n 
            for i in range(m)])

In [16]:
assert U.dimensions() == (m, m), "dimensions of U don't match: " + str(U.dimensions())
assert S.dimensions() == (m, n), "dimensions of S don't match: " + str(S.dimensions())
assert V.dimensions() == (n, n), "dimensions of V don't match: " + str(V.dimensions())

In [17]:
U, S, V

(
[ 1/2*sqrt(2)  1/2*sqrt(2)]  [5 0]  [ 1/2*sqrt(2)  1/2*sqrt(2)]
[-1/2*sqrt(2)  1/2*sqrt(2)], [0 1], [-1/2*sqrt(2)  1/2*sqrt(2)]
)

In [18]:
U * S * Vt, (U * S * Vt) == A

(
[ 3 -2]      
[-2  3], True
)

Comparison with sage implementation of SVD. (only available on RealDoubleField matrices)

In [19]:
Ar = matrix([[RDF(A[i,j]) for j in range(A.dimensions()[1])] for i in range(A.dimensions()[0])])
Ur, Sr, Vr = Ar.SVD()
Ur, Sr, Vr

(
[-0.7071067811865476  0.7071067811865474]
[ 0.7071067811865476  0.7071067811865477],

[ 4.999999999999999                0.0]
[               0.0 1.0000000000000002],

[-0.7071067811865476  0.7071067811865474]
[ 0.7071067811865474  0.7071067811865476]
)

In [20]:
Ur * Sr * Vr

[                3.0 -1.9999999999999991]
[               -2.0   2.999999999999999]